In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, Dense, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.layers import Dropout
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,plot_confusion_matrix,ConfusionMatrixDisplay,multilabel_confusion_matrix

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K# Creating Densenet121def densenet(input_shape, n_classes, filters = 32):



# Creating Densenet121def densenet(input_shape, n_classes, filters = 32):
def densenet(input_shape, n_classes, filters = 32):
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):

        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x

    def dense_block(x, repetition):

        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x

    def transition_layer(x):

        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x

    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)

    for repetition in [6,12,24,16]:

        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)

    model = Model(input, output)
    return model
input_shape = 224, 224, 3
n_classes = 3
model = densenet(input_shape,n_classes)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                            

In [ ]:
import tensorflow as tf
import tensorflow.keras as K

def preprocess_data(X, Y):

    X_p = K.applications.densenet.preprocess_input(X)

    Y_p = K.utils.to_categorical(Y, 10)

    return X_p, Y_p


(X_train_cifar, y_train_cifar), (X_val_cifar, y_val_cifar) = K.datasets.cifar10.load_data()

X_train_cifar, y_train_cifar = preprocess_data(X_train_cifar, y_train_cifar)

X_val_cifar, y_val_cifar = preprocess_data(X_val_cifar, y_val_cifar)


X_train_cifar = X_train_cifar[0:10000]
y_train_cifar = y_train_cifar[0:10000]
X_val_cifar = X_val_cifar[0:2000]
y_val_cifar = y_val_cifar[0:2000]


initializer = K.initializers.he_normal(seed=32)

input_shape_densenet = (224, 224, 3)

densenet_model = K.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape_densenet,
    pooling=None
)


densenet_model.trainable = False

for layer in densenet_model.layers:
#  if 'conv5' in layer.name:
#    layer.trainable = True
#  else:
    layer.trainable = False

densenet_model.summary()

input = K.Input(shape=(32, 32, 3))
preprocess = K.layers.Lambda(lambda x: tf.image.resize(x, (224, 224)), name='lamb')(input)

layer = densenet_model(inputs=preprocess)

layer = K.layers.Flatten()(layer)

layer = K.layers.BatchNormalization()(layer)

layer = K.layers.Dense(units=16,
                        activation='relu',
                        kernel_initializer=initializer
                        )(layer)

layer = K.layers.Dropout(0.4)(layer)

layer = K.layers.BatchNormalization()(layer)

layer = K.layers.Dense(units=10,
                       activation='softmax',
                       kernel_initializer=initializer
                       )(layer)

model = K.models.Model(inputs=input, outputs=layer)

model.summary()

model.compile(loss='binary_crossentropy',
            optimizer=K.optimizers.Adam(),
            metrics=['accuracy'])


history = model.fit(X_train_cifar, y_train_cifar, epochs=20, validation_data=(X_val_cifar, y_val_cifar), batch_size=32, verbose=1)

model.save('cifar10.h5')

29084464/29084464 [==============================] - 2s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       